In [2]:
import SimpleITK as sitk 
import numpy as np 
from dicom_to_cnn.model.post_processing.mip.MIP_Generator import MIP_Generator
from dicom_to_cnn.tools.visualization.create_mip import *

path_pet = r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_PT.nii'
path_mask= r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_mask_3d.nii'
pet_array = sitk.GetArrayFromImage(sitk.ReadImage(path_pet))
mask_array = sitk.GetArrayFromImage(sitk.ReadImage(path_mask))
print(pet_array.shape)
print(mask_array.shape)

#mip_generator = MIP_Generator(pet_array)
#mip = mip_generator.projection(0)
#mip_generator.save('t', r'C:\Users\wendy\Documents\STAGE\PET0', vmin=0, vmax=7 )
#mip_generator.create_mip_gif('t_gif', r'C:\Users\wendy\Documents\STAGE\PET0', vmin=0, vmax=7)

save_projection_two_modality(pet_array, mask_array, angle=0, filename='2t', directory=r'C:\Users\wendy\Documents\STAGE\PET0', vmin=0, vmax=7)
create_gif_two_modality(pet_array, mask_array, filename='2t_gif', directory=r'C:\Users\wendy\Documents\STAGE\PET0', vmin=0, vmax=7)

(287, 128, 128)
(287, 128, 128, 4)
